In [1]:
import os
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from IPython.display import clear_output
from keras.layers.pooling import AveragePooling2D
import joblib

In [2]:
datapath="/content/drive/MyDrive/Isro_project/Processing/Training"
outputmodel="/content/drive/MyDrive/Isro_project/Model/Model.sav"
outputlabbelbinarizer="/content/drive/MyDrive/Isro_project/Model/lblbinarizer"


In [11]:
label_isro=set()
for root,dir,files in  os.walk("/content/drive/MyDrive/Isro_project/Video Set"):
    for lb in dir:
        label_isro.add(lb)

In [12]:
label_isro

{'AerialView',
 'AirPlane',
 'Animation',
 'Building',
 'CompositionFrame',
 'Crowd',
 'DisplayScreen',
 'Forest',
 'Garden',
 'Graphics',
 'Helicopter',
 'Hospital',
 'IndoorControlRoom',
 'IndoorGeneric',
 'IndoorHome',
 'IndoorLab',
 'Interview',
 'Launch',
 'Logo',
 'Mountain',
 'OutdoorAntenna',
 'OutdoorGeneric',
 'OutdoorLaunchpad',
 'PersonCloseUp',
 'Satellite',
 'Sky',
 'Speech',
 'Text',
 'Traffic',
 'Vehicle'}

In [13]:
pathtoimages=list(paths.list_images(datapath))

In [14]:
len(pathtoimages)

8667

In [15]:
!nvidia-smi

Sat Jul 15 05:52:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
data=[]
labels=[]
img_count=len(pathtoimages)
count=0
for images in pathtoimages:
    count+=1
    label=images.split(os.path.sep)[-2]
    if label not in label_isro:
        continue
    print(label)
    image=cv2.imread(images)
    image=cv2.resize(image,(224,224))
    data.append(image)
    labels.append(label)
    print(f"Images Uploading : {round((count/img_count)*100,2)}%")
    clear_output(wait=1)

In [ ]:
joblib.dump(data,"/content/drive/MyDrive/Isro_project/data/data")
joblib.dump(labels,"/content/drive/MyDrive/Isro_project/data/labels")

['/content/drive/MyDrive/Isro_project/data/labels']

In [3]:
data=joblib.load("/content/drive/MyDrive/Isro_project/data/data")
labels=joblib.load("/content/drive/MyDrive/Isro_project/data/labels")

In [6]:
len(data)

8667

In [4]:
data=np.array(data)
labels=np.array(labels)
mlb=LabelBinarizer()
labels=mlb.fit_transform(labels)
labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [5]:
(x_train,x_test,y_train,y_test)=train_test_split(data,labels,test_size=0.20, stratify=labels, random_state=42)

In [6]:
train_augmentation=tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)
validation_aurgmentation=tf.keras.preprocessing.image.ImageDataGenerator()
mean=np.array([123.68,116.779,103.939],dtype="float32")
validation_aurgmentation.mean=mean
train_augmentation.mean=mean

In [7]:
basemodel=tf.keras.applications.ResNet50(weights="imagenet",include_top=False,input_tensor=tf.keras.layers.Input(shape=(224,224,3)))
headModel=basemodel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=tf.keras.layers.Flatten(name="flatten")(headModel)
headModel=tf.keras.layers.Dense(512,activation="relu")(headModel)
headModel=tf.keras.layers.Dropout(0.5)(headModel)
headModel=tf.keras.layers.Dense(len(mlb.classes_),activation="softmax")(headModel)
model=tf.keras.models.Model(inputs=basemodel.input,outputs=headModel)

for basemodellayer in basemodel.layers:
    basemodellayer.trainable=False

In [13]:
model=joblib.load(outputmodel)

In [8]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

In [9]:
History=model.fit(train_augmentation.flow(x_train,y_train,batch_size=32),
                  steps_per_epoch=len(x_train)//32,
                  validation_data=validation_aurgmentation.flow(x_test,y_test),
                  validation_steps=len(x_test)//32,
                  epochs=10) # Epochs should be atleast 70 to reduce loss and increase accuracy to reduce time i am using 10

Epoch 1/10
216/216 [==============================] - 105s 425ms/step - loss: 1.5236 - accuracy: 0.5841 - val_loss: 1.0051 - val_accuracy: 0.7106
Epoch 2/10
216/216 [==============================] - 94s 434ms/step - loss: 0.9878 - accuracy: 0.7122 - val_loss: 0.7796 - val_accuracy: 0.7784
Epoch 3/10
216/216 [==============================] - 95s 442ms/step - loss: 0.8350 - accuracy: 0.7502 - val_loss: 0.7368 - val_accuracy: 0.7899
Epoch 4/10
216/216 [==============================] - 90s 417ms/step - loss: 0.7480 - accuracy: 0.7747 - val_loss: 0.6701 - val_accuracy: 0.8079
Epoch 5/10
216/216 [==============================] - 93s 432ms/step - loss: 0.6712 - accuracy: 0.7966 - val_loss: 0.5776 - val_accuracy: 0.8328
Epoch 6/10
216/216 [==============================] - 89s 412ms/step - loss: 0.5845 - accuracy: 0.8144 - val_loss: 0.5514 - val_accuracy: 0.8449
Epoch 7/10
216/216 [==============================] - 93s 432ms/step - loss: 0.5517 - accuracy: 0.8277 - val_loss: 0.5804 - val_a

In [10]:
joblib.dump(model,outputmodel)
joblib.dump(mlb,outputlabbelbinarizer)

['/content/drive/MyDrive/Isro_project/Model/lblbinarizer']

In [11]:
print(model)

In [12]:
model.evaluate(x_test,y_test)

55/55 [==============================] - 7s 106ms/step - loss: 0.4681 - accuracy: 0.8731


[0.4680982828140259, 0.8731257319450378]